In [1]:
import boto3
import os

key_id = os.environ.get("AWS_ACCESS_KEY_ID")
secret_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
bucket_name = os.environ.get("AWS_S3_BUCKET")
s3_endpoint = os.environ.get("AWS_S3_ENDPOINT")

s3_client = boto3.client("s3", aws_access_key_id=key_id, aws_secret_access_key=secret_key, endpoint_url=s3_endpoint)

In [2]:
! mkdir model
s3_client.download_file(bucket_name, "/model/MLP_Keras_e19.h5", './model/model.h5')

mkdir: cannot create directory ‘model’: File exists


In [3]:
!pip install tf2onnx
!python -m tf2onnx.convert --saved-model ./model/saved_model/ --output ./model/model.onnx --opset 13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.0 MB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.6 MB/s eta 0:00:00


  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6


    Uninstalling protobuf-3.19.6:


      Successfully uninstalled protobuf-3.19.6


...

/usr/lib64/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2023-02-17 10:20:21.181027: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-17 10:20:21.181061: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-17 10:20:21.181084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (model-0): /proc/driver/nvidia/version does not exist
2023-02-17 10:20:21.181340: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the 

2023-02-17 10:20:21,340 - INFO - Signatures found in model: [serving_default].
2023-02-17 10:20:21,340 - WARNING - '--signature_def' not specified, using first signature: serving_default
2023-02-17 10:20:21,340 - INFO - Output names: ['dense_2']
2023-02-17 10:20:21.341681: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


2023-02-17 10:20:21.397891: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


2023-02-17 10:20:21,419 - INFO - Using tensorflow=2.11.0, onnx=1.13.0, tf2onnx=1.13.0/2c1db5
2023-02-17 10:20:21,419 - INFO - Using opset <onnx, 13>
2023-02-17 10:20:21,427 - INFO - Computed 0 values for constant folding
2023-02-17 10:20:21,437 - INFO - Optimizing ONNX model


2023-02-17 10:20:21,472 - INFO - After optimization: Identity -2 (2->0)
2023-02-17 10:20:21,475 - INFO - 
2023-02-17 10:20:21,475 - INFO - Successfully converted TensorFlow model ./model/saved_model/ to ONNX
2023-02-17 10:20:21,475 - INFO - Model inputs: ['dense_input']
2023-02-17 10:20:21,475 - INFO - Model outputs: ['dense_2']
2023-02-17 10:20:21,475 - INFO - ONNX model is saved at ./model/model.onnx


In [4]:
s3_client.upload_file("./model/model.onnx", bucket_name, "/onnx/model.onnx")

In [5]:
! rm -rf model